# Function Calling

Function calling allows you to define functions (with schemas) that the model can call as part of its response, enabling structured outputs, tool use, and more advanced workflows. 

For us, this seems like a first step towards our goal of enabling more complex interactions with LLMs, where they can not only generate text but also perform actions based on that text.

## Using OpenAI

### Prerequisites

Make sure you have your OpenAI API key set up as described in the previous notebook. We'll use the Python SDK for these examples.

In [1]:
# VS Code's Jupyter extension doesn't support loading .envrc, so if you're using VS Code, we load it here.

from utils import load_envrc

load_envrc()

### Defining a function schema

We need to define a function that the model can use. Here we'll use a silly example function that "cactifies" a name. The schema for the function is defined in [JSON Schema format](https://json-schema.org/docs) and looks like this:

```json
{
  "name": "cactify_name",
  "type": "function",
  "description": "Transforms a name into a fun, cactus-themed version.",
  "parameters": {
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "The name to be cactified."
      }
    },
    "required": ["name"]
  }
}
```

Following the [OpenAI function calling docs](https://platform.openai.com/docs/guides/function-calling), we can pass a `tools` parameter to the API endpoint to define functions the model can call. Here we use `curl` to demonstrate the raw API request:

In [2]:
%%bash --out curl_response

curl https://api.openai.com/v1/responses -s \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-5-nano",
    "input": [
      {"role": "user", "content": "What would my name, Colin, be if it were cactus-ified?"}
    ],
    "tools": [
      {
        "name": "cactify_name",
        "type": "function",
        "description": "Transforms a name into a fun, cactus-themed version.",
        "parameters": {
          "type": "object",
          "properties": {
            "name": {
              "type": "string",
              "description": "The name to be made cactus-like."
            }
          },
          "required": ["name"]
        }
      }
    ]
  }'

In [3]:
import json

from rich import print as rich_print

data = json.loads(curl_response)  # noqa
rich_print(data["output"])

[
    {'id': 'rs_0788c68d00cf12110068dd1753989c8196b6ba22ab5b539425', 'type': 'reasoning', 'summary': []},
    {
        'id': 'fc_0788c68d00cf12110068dd17551aa881968e5de7b4b0d2a5c9',
        'type': 'function_call',
        'status': 'completed',
        'arguments': '{"name":"Colin"}',
        'call_id': 'call_Zbe1dnFFQ9TBey3ym4pxAweK',
        'name': 'cactify_name'
    }
]

Reviewing the output, we see that the model decided to call our `cactify_name` function with the argument `"Colin"`. The model itself doesn't actually execute the function. It simply returns the function call in its response. It's up to us to handle the function execution and return the result if needed.

It's easier to see this in action using the Python SDK, which we'll explore next.

### Using the Python SDK

Now let's see how to do the same thing using the Python SDK. We'll define the function schema as a variable:

In [4]:
cactify_name_schema = {
    "name": "cactify_name",
    "type": "function",
    "description": "Transform a name into a fun, cactus-themed version.",
    "parameters": {
        "type": "object",
        "properties": {"name": {"type": "string", "description": "The name to be cactified."}},
        "required": ["name"],
    },
}

### Making a function call request

Now, let's ask the model to cactify a name using the function we defined:

In [5]:
import openai

client = openai.Client()

input_list = [{"role": "user", "content": "What would my name, Colin, be if it were cactus-ified?"}]
tools = [cactify_name_schema]

response = client.responses.create(
    model="gpt-5-nano",
    input=input_list,
    tools=tools,
)
rich_print(response)

Response(
    id='resp_070610056460d7a00068dd1757c584819099b1d6c7af2a9bed',
    created_at=1759319895.0,
    error=None,
    incomplete_details=None,
    instructions=None,
    metadata={},
    model='gpt-5-nano-2025-08-07',
    object='response',
    output=[
        ResponseReasoningItem(
            id='rs_070610056460d7a00068dd1758a0b881908a4d3ed8b687918f',
            summary=[],
            type='reasoning',
            content=None,
            encrypted_content=None,
            status=None
        ),
        ResponseFunctionToolCall(
            arguments='{"name":"Colin"}',
            call_id='call_I7BVpmNmDnbYnv7JZOT1srtC',
            name='cactify_name',
            type='function_call',
            id='fc_070610056460d7a00068dd1759a49c819089ad17329ab2013f',
            status='completed'
        )
    ],
    parallel_tool_calls=True,
    temperature=1.0,
    tool_choice='auto',
    tools=[
        FunctionTool(
            name='cactify_name',
            parameters={
                'type': 'object',
                'properties': {'name': {'type': 'string', 'description': 'The name to be cactified.'}},
                'required': ['name'],
                'additionalProperties': False
            },
            strict=True,
            type='function',
            description='Transform a name into a fun, cactus-themed version.'
        )
    ],
    top_p=1.0,
    background=False,
    conversation=None,
    max_output_tokens=None,
    max_tool_calls=None,
    previous_response_id=None,
    prompt=None,
    prompt_cache_key=None,
    reasoning=Reasoning(effort='medium', generate_summary=None, summary=None),
    safety_identifier=None,
    service_tier='default',
    status='completed',
    text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'),
    top_logprobs=0,
    truncation='disabled',
    usage=ResponseUsage(
        input_tokens=73,
        input_tokens_details=InputTokensDetails(cached_tokens=0),
        output_tokens=151,
        output_tokens_details=OutputTokensDetails(reasoning_tokens=128),
        total_tokens=224
    ),
    user=None,
    billing={'payer': 'developer'},
    store=True
)

Like the `curl` example, the response includes a function call with arguments. Let's extract and run it.

In [6]:
def cactify_name(name: str) -> str:
    """
    Makes a name more cactus-like.

    Args:
        name: The name to be cactified.

    Returns:
        The cactified version of the name.
    """

    base_name = name

    # Rule 1: If the name ends in 's' or 'x', remove it.
    # Example: "James" -> "Jame", "Alex" -> "Ale"
    if base_name.lower().endswith(("s", "x")):
        base_name = base_name[:-1]

    # Rule 2: If the name now ends in a vowel, remove it.
    # Example: "Jame" -> "Jam", "Mike" -> "Mik", "Anna" -> "Ann"
    if base_name and base_name.lower()[-1] in "aeiou":
        base_name = base_name[:-1]

    # Add the smoother suffix
    return base_name + "actus"

In [7]:
import json

response_output = response.output
# The model decided to call our function
function_call = response.output[1]
# Load the arguments provided by the model to call the function
args = json.loads(function_call.arguments)
result = cactify_name(**args)
print(f"Result: {result}")

Result: Colinactus


Now that we have the output, we want to feed it back to the model to get a final response.

In [8]:
# Add the model's call to our function to the input list
input_list += response_output
# Append the function call output to the input list
input_list.append(
    {"type": "function_call_output", "call_id": function_call.call_id, "output": result}
)
input_list

[{'role': 'user',
  'content': 'What would my name, Colin, be if it were cactus-ified?'},
 ResponseReasoningItem(id='rs_070610056460d7a00068dd1758a0b881908a4d3ed8b687918f', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFunctionToolCall(arguments='{"name":"Colin"}', call_id='call_I7BVpmNmDnbYnv7JZOT1srtC', name='cactify_name', type='function_call', id='fc_070610056460d7a00068dd1759a49c819089ad17329ab2013f', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_I7BVpmNmDnbYnv7JZOT1srtC',
  'output': 'Colinactus'}]

In [9]:
response = client.responses.create(
    model="gpt-4o",
    instructions="Tell the user what their name would be if it were cactus-ified.",
    tools=tools,
    input=input_list,
)

In [10]:
rich_print(response.output[0].content[0])

ResponseOutputText(
    annotations=[],
    text='Your cactus-ified name would be Colinactus! 🌵',
    type='output_text',
    logprobs=[]
)

There we go! We've successfully used function calling with the Python SDK to cactify a name. It's a bit silly, but it shows how function calling can be used.

### Detecting function calls and saving history

We manually called the function and fed the output back to the model. In a real application, you'd want to automate this process. You could write a loop that checks if the model's response includes a function call, executes the function, and then sends the result back to the model until you get a final text response.

In [11]:
all_messages = []


def prompt(user_input: str) -> str:
    """Prompt the model with the user input."""

    # Add the user input to the conversation history
    all_messages.append({"role": "user", "content": user_input})
    # Prompt the model with the user input
    response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=all_messages,
    )

    for event in response.output:
        all_messages.append(event)

        # There's a request from the model to use a tool
        if event.type == "function_call":
            function_name = event.name
            function_args = json.loads(event.arguments)

            # Execute the function based on its name
            if function_name == "cactify_name":
                result = cactify_name(function_args["name"])

            # Add the function call output to the all_messages list
            # Use the exact format expected by the API
            all_messages.append(
                {
                    "type": "function_call_output",
                    "call_id": event.call_id,
                    "output": json.dumps(result),
                }
            )

            # Now feed the function result back to the model
            final_response = client.responses.create(
                model="gpt-5-nano",
                instructions="Respond with what the name would be if it were cactus-ified in a sentence.",
                tools=tools,
                input=all_messages,
            )

            for final_event in final_response.output:
                if final_event.type == "message":
                    text = final_event.content[0].text
                    all_messages.append({"role": "assistant", "content": text})
                    return text

        elif event.type == "message":
            text = event.content[0].text
            all_messages.append({"role": "assistant", "content": text})
            return text

In [12]:
print(prompt("What would my name, Colin, be if it were cactus-ified?"))

Colin would be Colinactus.


In [13]:
print(prompt("What about Simon?"))

Simon would be Simonactus.


In [14]:
print(prompt("What names did I ask about"))

Colin and Simon.


## Using local models with Ollama

Not all Ollama models support function-calling, but we can find one that does by [filtering by "tools" on the Ollama search page](https://ollama.com/search?c=tools).

Let's try the `llama3.2` model. If you haven't done so already, [install Ollama](https://ollama.com/download) on your computer and download the model with `ollama pull llama3.2`.

### Defining a function schema

We'll define our function using JSON Schema format. The schema is similar to what we defined for the OpenAI example, but slightly different:
```json
{
  "type": "function",
  "function": {
    "name": "cactify_name",
    "description": "Transforms a name into a fun, cactus-themed version.",
    "parameters": {
      "type": "object",
      "properties": {
        "name": {
          "type": "string",
          "description": "The name to be cactified."
        }
      },
      "required": ["name"]
    }
  }
}
```

### Using `curl`

Ollama's API is available at `http://localhost:11434` by default. Let's use `curl` to make a request to the `/api/chat/` endpoint and see the raw JSON response from the model. We'll provide our function schema in the `tools` parameter.

In [15]:
%%bash --out curl_response

curl http://localhost:11434/api/chat -s -d '{
  "model": "llama3.2",
  "messages": [
    {
      "role": "user",
      "content": "What would my name, Colin, be if it were cactus-ified?"
    }
  ],
  "stream": false,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "cactify_name",
        "description": "Transforms a name into a fun, cactus-themed version.",
        "parameters": {
          "type": "object",
          "properties": {
            "name": {
              "type": "string",
              "description": "The name to be cactified."
            }
          },
          "required": ["name"]
        }
      }
    }
  ]
}'

In [16]:
import json

from rich import print as rich_print

data = json.loads(curl_response)  # noqa
rich_print(data)

{
    'model': 'llama3.2',
    'created_at': '2025-10-01T11:58:41.961900899Z',
    'message': {
        'role': 'assistant',
        'content': '',
        'tool_calls': [{'function': {'name': 'cactify_name', 'arguments': {'name': 'Colin'}}}]
    },
    'done_reason': 'stop',
    'done': True,
    'total_duration': 7474827644,
    'load_duration': 51701877,
    'prompt_eval_count': 184,
    'prompt_eval_duration': 5693575941,
    'eval_count': 20,
    'eval_duration': 1728709049
}

We see that the model decided to make a function call. As with the OpenAI example, the model simply tells us which function it would like to call and with which arguments. It's up to us to handle the function execution and pass back the result.

### Using the `ollama` Python library

We can make the same request using Python.

In [17]:
import ollama

input_list = [{"role": "user", "content": "What would my name, Colin, be if it were cactus-ified?"}]
cactify_name_schema = {
    "type": "function",
    "function": {
        "name": "cactify_name",
        "description": "Transforms a name into a fun, cactus-themed version.",
        "parameters": {
            "type": "object",
            "properties": {"name": {"type": "string", "description": "The name to be cactified."}},
            "required": ["name"],
        },
    },
}
tools = [cactify_name_schema]

response = ollama.chat(
    "llama3.2",
    messages=input_list,
    tools=tools,
)

rich_print(response)

ChatResponse(
    model='llama3.2',
    created_at='2025-10-01T11:58:44.12875799Z',
    done=True,
    done_reason='stop',
    total_duration=2088280143,
    load_duration=61502657,
    prompt_eval_count=184,
    prompt_eval_duration=98568995,
    eval_count=20,
    eval_duration=1927421880,
    message=Message(
        role='assistant',
        content='',
        thinking=None,
        images=None,
        tool_name=None,
        tool_calls=[ToolCall(function=Function(name='cactify_name', arguments={'name': 'Colin'}))]
    )
)

We can also pass the actual Python function in the `tools` argument and Ollama will generate the schema for us in the background. This makes it easy to use an existing function as a tool. For best results, it's recommended to provide type annotations for parameters and return values, and add a [Google-style docstring](https://google.github.io/styleguide/pyguide.html#doc-function-raises). We've already done this for our `cactify_name` function.

In [18]:
response = ollama.chat(
    "llama3.2",
    messages=input_list,
    tools=[cactify_name],
)

rich_print(response)

ChatResponse(
    model='llama3.2',
    created_at='2025-10-01T11:58:49.688111262Z',
    done=True,
    done_reason='stop',
    total_duration=5545610881,
    load_duration=69897155,
    prompt_eval_count=179,
    prompt_eval_duration=3538477099,
    eval_count=20,
    eval_duration=1936451125,
    message=Message(
        role='assistant',
        content='',
        thinking=None,
        images=None,
        tool_name=None,
        tool_calls=[ToolCall(function=Function(name='cactify_name', arguments={'name': 'Colin'}))]
    )
)

Let's execute the function call.

In [19]:
function_call = response.message.tool_calls[0].function
# The arguments are already a Python dict, not JSON
result = cactify_name(**function_call.arguments)
print(f"Result: {result}")

Result: Colinactus


Then we'll feed back the result to the model to get a final response.

In [20]:
# Add the model's response to the input_list first, for conversation history
input_list.append(response.message)
input_list.append({"role": "tool", "content": result})
input_list

[{'role': 'user',
  'content': 'What would my name, Colin, be if it were cactus-ified?'},
 Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='cactify_name', arguments={'name': 'Colin'}))]),
 {'role': 'tool', 'content': 'Colinactus'}]

In [21]:
response = ollama.chat(
    "llama3.2",
    messages=input_list,
    tools=tools,
)

rich_print(response)

ChatResponse(
    model='llama3.2',
    created_at='2025-10-01T11:58:54.744924972Z',
    done=True,
    done_reason='stop',
    total_duration=5035987618,
    load_duration=71600567,
    prompt_eval_count=105,
    prompt_eval_duration=3135668337,
    eval_count=19,
    eval_duration=1826394100,
    message=Message(
        role='assistant',
        content='The cactified version of your name, Colin, is indeed "Colinactus".',
        thinking=None,
        images=None,
        tool_name=None,
        tool_calls=None
    )
)

### Detecting function calls and saving history

Let's create a function that automates the whole process of prompting, detecting a function call, executing the function call, and sending it's result back to the model.

In [41]:
ollama_messages = []


def prompt_local(user_input: str) -> str:
    """Prompt the model with the user input."""

    # Add the user input to the conversation history
    ollama_messages.append({"role": "user", "content": user_input})
    # Prompt the model with the user input
    response = ollama.chat(
        "llama3.2",
        messages=ollama_messages,
        tools=tools,
    )

    if response.message.tool_calls:
        # There's a request from the model to use one or more tools
        ollama_messages.append(response.message)

        for tool_call in response.message.tool_calls:
            # Execute the function based on its name
            if tool_call.function.name == "cactify_name":
                result = cactify_name(**tool_call.function.arguments)
                # Add the function call output to the messages list
                ollama_messages.append(
                    {"role": "tool", "content": result, "tool_name": "cactify_name"}
                )

        # Now feed the function result back to the model
        final_response = ollama.chat("llama3.2", messages=ollama_messages, tools=tools)

        ollama_messages.append(final_response.message)

        return final_response.message.content

    return response.message.content

In [42]:
print(prompt_local("What would my name, Colin, be if it were cactus-ified?"))

Based on the tool call response, I've formed an answer to your original question: If Colin's name were cactus-ified, it would be Colinactus.


In [43]:
print(prompt_local("What about Simon?"))

Based on the tool call response, I've formed an answer to your original question: If Simon's name were cactus-ified, it would be Simonactus.


In [44]:
print(prompt_local("What names did I ask about?"))

Based on our conversation, you asked about cactifying the names Colin and Simon. The resulting names are Colinactus and Simonactus, respectively.


I found that sometimes the model would ignore the result of the function call and come up with its own cactus-ified name. It would also sometimes attempt to call the function with incorrect arguments. For example, on the "What about Simon?" prompt, it would specify the function argument to be "Simonactus", maybe basing it on the previous prompt's result. Or on "What names did I ask about?", it would try to call the function with a JSON array like `["Colin", "Simon"]`. The results may be better or more consistent with larger models.